Tenemos un RDD con información de recetas:

(ID_Receta, Nombre, Categoría)

Y otro RDD con los ingredientes de cada receta:

(ID_Receta, Ingrediente, Cantidad_Kg)

Queremos obtener:

a) Listar todos los ingredientes que aparecen en alguna receta que usa "pollo" indicando en
cuantas recetas el ingrediente y pollo aparecen juntos. El formato de salida es (ingrediente,
cantidad de recetas en que aparece junto con pollo). Por ejemplo, la papa aparece en 10
recetas con pollo, por lo que tendríamos (papa, 10). (50 pts)

b) Queremos obtener todos los nombres de recetas Mediterráneas que no tengan ni papa ni
pollo entre sus ingredientes.(50 pts)

Resolver los puntos usando la API de RDDs de PySpark


In [7]:
import pyspark as ps
import pandas as pd

spark = ps.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

#Leo los csv que genere de dummy
recetasRdd = spark.read.csv('/home/marcos/PycharmProjects/PracticaSpark/parcialRecetas.csv', header=True).rdd
ingredientesRdd=spark.read.csv('/home/marcos/PycharmProjects/PracticaSpark/parcialIngredientes.csv',header=True).rdd

#Las siguientes 2 lineas son para pasar de texto al dato correspondiente del csv que lei
ingredientesRdd = ingredientesRdd.map(lambda x: (pd.to_numeric(x[0], 'coerce'), x[1],
                                                 pd.to_numeric(x[2], 'coerce')))
recetasRdd = recetasRdd.map(lambda x: (pd.to_numeric(x[0], 'coerce'), x[1], x[2]))

#A partir de aca arranca la resolucion en cuestion
ingredientesRdd = ingredientesRdd.map(lambda x: (x[0], x[1])).cache() 
                                            #agregar .distinct() antes de .cache() si hubiese
                                            #por algun motivo receta + ingrediente repetido
                                            #como [(r1, pollo, 10), (r1, pollo, 20)]
                                            #El enunciado no aclara (y no seria muy logico)
                                            #asi que asumo que no es el caso

recetasConPollo = ingredientesRdd.filter(lambda x: x[1] == 'pollo')

ingredientesRdd.filter(lambda x: x[1] != 'pollo').join(recetasConPollo) \
        .map(lambda x: (x[1][0], 1)).reduceByKey(lambda x,y: x + y).collect() 
                                        #hago collect porque entiendo que quieren que muestre todos

[('papa', 115), ('choclo', 115), ('queso', 100)]

In [8]:
recetasSinPapaNiPollo = ingredientesRdd.map(lambda x: (x[0], 1 if ((x[1] == 'pollo') 
                        | (x[1] == 'papa')) else 0)).reduceByKey(lambda x,y: x + y) \
                        .filter(lambda x: x[1] == 0)

recetasRdd.filter(lambda x: x[2] == 'Mediterranea').map(lambda x: (x[0], x[1])) \
        .join(recetasSinPapaNiPollo).map(lambda x: x[1][0]).collect() 
                                #hago collect porque entiendo que quieren que muestre todos

['pciujpqaam', 'sukwlcbko']